### 1. Importing the libraries

In [2]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup 
import requests 
!pip install folium
import folium 

!import json 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install geopy
from geopy.geocoders import Nominatim

     |████████████████████████████████| 92kB 13.1MB/s eta 0:00:01
/usr/bin/sh: import: command not found


### 2. Scrape website for Neighbourhoods

In [3]:
url = 'https://en.wikipedia.org/wiki/Boroughs_of_Amsterdam'
response = requests.get(url)
#response.text
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find_all('table', class_ = 'wikitable sortable' )

table = soup.find_all('table')

df = pd.read_html(str(table))[1]
df = df.drop(['Location (in green)'], axis=1)

df2 = df.Borough.apply(lambda x: pd.Series(str(x).split("(")))
df2 = df2[1].apply(lambda x: pd.Series(str(x).replace(")","")))

df3 = df['Neighbourhoods'].apply(lambda x: pd.Series(str(x).replace(" en " , ", ")))

df['Borough'] = df2[0]
df['Neighbourhoods'] = df3[0]

df.head(8)

,Borough,Area,Population,Population density,Neighbourhoods
0,Centre,8.04 km²,86422,"13,748/km²","Binnenstad, Grachtengordel, Haarlemmerbuurt, J..."
1,North,49.01 km²,94766,"2,269/km²","Banne Buiksloot, Buiksloot, Buikslotermeer, Fl..."
2,New West,32.38 km²,151677,"4,478/km²","Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O..."
3,East,30.56 km²,135767,"7,635/km²","IJburg, Indische Buurt, Eastern Docklands, Oud..."
4,nan,9.89 km²,143842,"15,252/km²","Frederik Hendrikbuurt, Houthaven, Spaarndammer..."
5,West Gateway,10 km²,192,10/km²,Westpoort
6,South,17.41 km²,144432,"9,349/km²","Apollobuurt, Buitenveldert, Hoofddorppleinbuur..."
7,Southeast,22.08 km²,87854,"4,391/km²","Bijlmermeer, Venserpolder, Gaasperdam, Driemond"


##### Cant find all of geocoordinates so creating a lookup for 5 Neighbourhoods

In [31]:
data = {'Neighbourhood':['Binnenstad','Jodenbuurt','Westelijke Eilanden','Overtoombuurt','Hoofddorppleinbuurt','Prinses Irenebuurt'],
        'Latitude':     [52.3733501,52.3688696,52.3855725,52.3635177,52.3480617,52.3416063],
        'Longitude':    [4.8864617,4.9002341,4.8840947,4.8629492,4.8393713,4.8513486]
        }

additionalgeo = pd.DataFrame (data, columns = ['Neighbourhood','Latitude','Longitude'])
additionalgeo.head()

,Neighbourhood,Latitude,Longitude
0,Binnenstad,52.373350,4.886462
1,Jodenbuurt,52.368870,4.900234
2,Westelijke Eilanden,52.385573,4.884095
3,Overtoombuurt,52.363518,4.862949
4,Hoofddorppleinbuurt,52.348062,4.839371


#### Add geo data to the neighbourhoods 

In [5]:
lat = []
lon = []

new_df = pd.DataFrame(df.Neighbourhoods.str.split(',').tolist(), index=df.Borough).stack()
new_df = new_df.reset_index([0, 'Borough'])

new_df.columns = ['Borough', 'Neighbourhood']
new_df['Neighbourhood'] = new_df['Neighbourhood'].apply(lambda x: pd.Series(str(x).lstrip()))

geolocator = Nominatim(user_agent="courseracourse")

for b in new_df['Neighbourhood']:
    locationname = b + ", Amsterdam, Nederland"
    location = geolocator.geocode(locationname)
    if(location is not None):
        lat.append(location.latitude)
        lon.append(location.longitude)
    else:
        r1 = additionalgeo[additionalgeo['Neighbourhood'] == b]
        lat.append(float(r1.Latitude))
        lon.append(float(r1.Longitude))

new_df['Latitude'] = lat
new_df['Longitude']= lon

new_df.head(8)    
 

,Borough,Neighbourhood,Latitude,Longitude
0,Centre,Binnenstad,52.373350,4.886462
1,Centre,Grachtengordel,52.370837,4.885478
2,Centre,Haarlemmerbuurt,52.382441,4.887193
3,Centre,Jodenbuurt,52.368870,4.900234
4,Centre,Jordaan,52.375416,4.881096
5,Centre,Kadijken,52.367753,4.919543
6,Centre,Lastage,52.371250,4.905508
7,Centre,Oosterdokseiland,52.376591,4.907560


#### Check number of Neighbourhood found

In [6]:

index = new_df.index
number_of_rows = len(index)
print(number_of_rows)

71


#### Draw all Neighbourhood on map

In [80]:
# get from google maps
amsterdam_latitude = 52.370216
amsterdam_longitude = 4.895168

# create map of ams using latitude and longitude values
map_amsterdam = folium.Map(location=[amsterdam_latitude, amsterdam_longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

#### Foursquare creds

In [8]:
CLIENT_ID = 'secret' # your Foursquare ID
CLIENT_SECRET = 'secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: secret
CLIENT_SECRET:secret


#### Function that extracts the category of the venue

In [9]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
LIMIT = 100
RADIUD = 500

In [12]:
venues = getNearbyVenues(names=new_df['Neighbourhood'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )

Binnenstad
Grachtengordel
Haarlemmerbuurt
Jodenbuurt
Jordaan
Kadijken
Lastage
Oosterdokseiland
Oostelijke Eilanden
Plantage
Rapenburg
Uilenburg
Westelijke Eilanden
Weteringschans
Banne Buiksloot
Buiksloot
Buikslotermeer
Floradorp
Kadoelen
Molenwijk
Nieuwendam
Nieuwendammerdijk
Buiksloterdijk
Oostzanerwerf
Overhoeks
Tuindorp Nieuwendam
Tuindorp Oostzaan
Geuzenveld
Nieuw Sloten
Oostoever
Osdorp
Overtoomse Veld
Sloten
Slotermeer
Slotervaart
IJburg
Indische Buurt
Eastern Docklands
Oud-Oost
Watergraafsmeer
Frederik Hendrikbuurt
Houthaven
Spaarndammerbuurt
Staatsliedenbuurt
Zeeheldenbuurt
Westerpark
Kinkerbuurt
Overtoombuurt
De Baarsjes
Bos
Lommer
Kolenkitbuurt
Landlust
Sloterdijk
Westpoort
Apollobuurt
Buitenveldert
Hoofddorppleinbuurt
Museumkwartier
De Pijp
Prinses Irenebuurt
Rivierenbuurt
Schinkelbuurt
Stadionbuurt
Vondelpark
Willemspark
Zuidas
Bijlmermeer
Venserpolder
Gaasperdam
Driemond


#### Show the venues shape and part of data

In [13]:
print(venues.shape)
venues.head()

(2486, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Binnenstad,52.37335,4.886462,Pulitzer's Bar,52.372553,4.884640,Hotel Bar
1,Binnenstad,52.37335,4.886462,De Bierkoning,52.372404,4.889795,Beer Store
2,Binnenstad,52.37335,4.886462,Jansz.,52.372427,4.884262,French Restaurant
3,Binnenstad,52.37335,4.886462,Puccini Bomboni,52.374592,4.889032,Chocolate Shop
4,Binnenstad,52.37335,4.886462,Hotel Pulitzer,52.372789,4.883466,Hotel


In [14]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Apollobuurt,19,19,19,19,19,19
Banne Buiksloot,12,12,12,12,12,12
Bijlmermeer,4,4,4,4,4,4
Binnenstad,66,66,66,66,66,66
Bos,55,55,55,55,55,55
Buiksloot,11,11,11,11,11,11
Buiksloterdijk,14,14,14,14,14,14
Buikslotermeer,23,23,23,23,23,23
Buitenveldert,47,47,47,47,47,47


#### Let's find out how many unique categories can be curated from all the returned venues

In [15]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 286 uniques categories.


#### What type of restaurants are in Amsterdam already

In [16]:
allvenues = venues['Venue Category'].unique()
#print(type(allvenues))

dfallvenues = pd.DataFrame(data=allvenues, columns=["Venue Category"])

dfallvenues.head()

,Venue Category
0,Hotel Bar
1,Beer Store
2,French Restaurant
3,Chocolate Shop
4,Hotel


In [17]:
allrestaurants = dfallvenues[dfallvenues['Venue Category'].str.contains('Restaurant')]
sortedallrestaurants = allrestaurants.sort_values(by=['Venue Category'])
sortedallrestaurants.head(50)

,Venue Category
28,Afghan Restaurant
128,African Restaurant
207,Asian Restaurant
212,Australian Restaurant
275,Belgian Restaurant
198,Cantonese Restaurant
220,Caribbean Restaurant
31,Chinese Restaurant
204,Doner Restaurant
158,Dutch Restaurant


### Analyze Each Neighborhood

In [18]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
amsterdam_grouped = onehot.groupby('Neighborhood').mean().reset_index()
amsterdam_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Apollobuurt,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000
1,Banne Buiksloot,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bijlmermeer,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Binnenstad,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.000000,0.015152,0.015152,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bos,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.054545,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.018182,0.000000
5,Buiksloot,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Buiksloterdijk,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Buikslotermeer,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Buitenveldert,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.021277,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,De Baarsjes,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.017241,0.000000


#### Get vegetarian / vegan restaurants count

In [20]:
len(amsterdam_grouped[amsterdam_grouped["Vegetarian / Vegan Restaurant"] > 0])

17

#### Get total restaurants in Amsterdam count

In [21]:
#len(amsterdam_grouped[amsterdam_grouped['Venue Category'].str.contains('Restaurant')]

In [22]:
totalrestaurants = 0

for sortedallrestaurant in sortedallrestaurants['Venue Category']:
    totalrestaurants = totalrestaurants + len(amsterdam_grouped[amsterdam_grouped[sortedallrestaurant] > 0])    

print(totalrestaurants)    

364


#### Get the percentage of veg restaurants

In [23]:
percentagevegrestaurant = (17/364) * 100
percentagevegrestaurant

4.670329670329671

#### Create new Dataframe        

In [43]:
ams_veg_restaurant = amsterdam_grouped[["Neighborhood","Vegetarian / Vegan Restaurant"]]
ams_veg_restaurant.head(10)

,Neighborhood,Vegetarian / Vegan Restaurant
0,Apollobuurt,0.000000
1,Banne Buiksloot,0.000000
2,Bijlmermeer,0.000000
3,Binnenstad,0.000000
4,Bos,0.054545
5,Buiksloot,0.000000
6,Buiksloterdijk,0.000000
7,Buikslotermeer,0.000000
8,Buitenveldert,0.000000
9,De Baarsjes,0.000000


#### Cluster the Neighborhoods


In [39]:
# set number of clusters
kclusters = 5

ams_veg_restaurant_cluster = ams_veg_restaurant.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ams_veg_restaurant_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int32)

#### Create a new dataframe that includes the cluster

In [45]:
ams_veg_restaurant_all_data = ams_veg_restaurant

ams_veg_restaurant_all_data.insert(0,'Cluster Labels', kmeans.labels_)
ams_veg_restaurant_all_data.head()

,Cluster Labels,Neighborhood,Vegetarian / Vegan Restaurant
0,0,Apollobuurt,0.000000
1,0,Banne Buiksloot,0.000000
2,0,Bijlmermeer,0.000000
3,0,Binnenstad,0.000000
4,1,Bos,0.054545


#### Naming is different in both dataframes so make it 1

In [46]:
goe_df = new_df.rename(columns={'Neighbourhood': 'Neighborhood'})

goe_df.head(15)

,Borough,Neighborhood,Latitude,Longitude
0,Centre,Binnenstad,52.373350,4.886462
1,Centre,Grachtengordel,52.370837,4.885478
2,Centre,Haarlemmerbuurt,52.382441,4.887193
3,Centre,Jodenbuurt,52.368870,4.900234
4,Centre,Jordaan,52.375416,4.881096
5,Centre,Kadijken,52.367753,4.919543
6,Centre,Lastage,52.371250,4.905508
7,Centre,Oosterdokseiland,52.376591,4.907560
8,Centre,Oostelijke Eilanden,52.371869,4.922875
9,Centre,Plantage,52.366405,4.913729


#### Add geocoordinates to dataframe

In [47]:
ams_merged = ams_veg_restaurant_all_data.join(goe_df.set_index('Neighborhood'), on='Neighborhood')
ams_merged.head(10)

,Cluster Labels,Neighborhood,Vegetarian / Vegan Restaurant,Borough,Latitude,Longitude
0,0,Apollobuurt,0.000000,South,52.348073,4.875559
1,0,Banne Buiksloot,0.000000,North,52.408505,4.918843
2,0,Bijlmermeer,0.000000,Southeast,52.317033,4.964991
3,0,Binnenstad,0.000000,Centre,52.373350,4.886462
4,1,Bos,0.054545,nan,52.358776,4.869654
5,0,Buiksloot,0.000000,North,52.392562,4.929279
6,0,Buiksloterdijk,0.000000,North,52.398198,4.923239
7,0,Buikslotermeer,0.000000,North,52.399439,4.943135
8,0,Buitenveldert,0.000000,South,52.328647,4.873523
9,0,De Baarsjes,0.000000,nan,52.368926,4.856382


#### Show results of clustering in folium map

In [81]:
# create map of ams using latitude and longitude values
map_veg_amsterdam = folium.Map(location=[amsterdam_latitude, amsterdam_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to map
for lat, lng, label, cluster in zip(ams_merged['Latitude'], ams_merged['Longitude'], ams_merged['Neighborhood'], ams_merged['Cluster Labels']):
        label = folium.Popup(str(label) + ' - Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7,
            parse_html=False).add_to(map_veg_amsterdam)  
    
map_veg_amsterdam

### Analyze cluster data

In [64]:
ams_merged[ams_merged['Cluster Labels'] == 0]

,Cluster Labels,Neighborhood,Vegetarian / Vegan Restaurant,Borough,Latitude,Longitude
0,0,Apollobuurt,0.0,South,52.348073,4.875559
1,0,Banne Buiksloot,0.0,North,52.408505,4.918843
2,0,Bijlmermeer,0.0,Southeast,52.317033,4.964991
3,0,Binnenstad,0.0,Centre,52.373350,4.886462
5,0,Buiksloot,0.0,North,52.392562,4.929279
6,0,Buiksloterdijk,0.0,North,52.398198,4.923239
7,0,Buikslotermeer,0.0,North,52.399439,4.943135
8,0,Buitenveldert,0.0,South,52.328647,4.873523
9,0,De Baarsjes,0.0,nan,52.368926,4.856382
11,0,Driemond,0.0,Southeast,52.305405,5.016549


In [75]:
ams_merged[ams_merged['Cluster Labels'] == 1]

,Cluster Labels,Neighborhood,Vegetarian / Vegan Restaurant,Borough,Latitude,Longitude
4,1,Bos,0.054545,nan,52.358776,4.869654
31,1,Lommer,0.040816,nan,52.378521,4.848738
62,1,Vondelpark,0.045977,South,52.357887,4.867117
67,1,Weteringschans,0.045455,Centre,52.360723,4.887778


In [76]:
ams_merged[ams_merged['Cluster Labels'] == 2]

,Cluster Labels,Neighborhood,Vegetarian / Vegan Restaurant,Borough,Latitude,Longitude
10,2,De Pijp,0.010000,South,52.354240,4.896946
18,2,Haarlemmerbuurt,0.015873,Centre,52.382441,4.887193
50,2,Schinkelbuurt,0.017857,South,52.352606,4.854591
57,2,Stadionbuurt,0.019231,South,52.346531,4.862066
64,2,Westelijke Eilanden,0.014925,Centre,52.385573,4.884095


In [77]:
ams_merged[ams_merged['Cluster Labels'] == 3]

,Cluster Labels,Neighborhood,Vegetarian / Vegan Restaurant,Borough,Latitude,Longitude
14,3,Frederik Hendrikbuurt,0.029412,nan,52.376956,4.874085
22,3,Indische Buurt,0.021277,East,52.361995,4.938969
27,3,Kinkerbuurt,0.030000,nan,52.369167,4.866649
42,3,Oud-Oost,0.023256,East,52.361400,4.930265
44,3,Overtoombuurt,0.021739,nan,52.363518,4.862949
49,3,Rivierenbuurt,0.022222,South,52.344040,4.903653
56,3,Staatsliedenbuurt,0.027778,nan,52.380286,4.870951


In [78]:
ams_merged[ams_merged['Cluster Labels'] == 4]

,Cluster Labels,Neighborhood,Vegetarian / Vegan Restaurant,Borough,Latitude,Longitude
29,4,Landlust,0.068966,nan,52.379851,4.858608


#### Conclusion: Seems that cluster 0 would be a good starting point of a new vegan or vegetarian restaurant.